In [ ]:
# For data manipulation
import numpy as np
import pandas as pd
import scipy.signal as signal

# For visualizing the data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
sns.set_style('darkgrid')

# For data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize


import sys
sys.path.append('../Librerias')
import dataset as ds
#no sé porqué no funciona cuando lo tengo en la carpeta de clusters
#pero si funciona cuando esta solo en la de notebooks,,, raro.

In [ ]:
#load data

fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/"
fname = "datosconsu_021023_bajos.mat"
cutoff = [8/1000, 11/1000]

set =  ds.MatFileToDataFrame(fpath, fname)
df = set.get_dataframe(cutoff)
df.tail()

In [ ]:
amplitude_env = np.abs(df['Hilbert Transform'])
inst_phase =  np.unwrap(np.angle(df['Hilbert Transform']))
inst_freq = (np.diff(inst_phase)/(2.0*np.pi)*1000)
diff_phase =  np.diff(inst_phase)
mean_phase = np.mean(diff_phase)
diff_phase = np.insert(diff_phase, 0, 0)  
df['Amplitude Envelope'] = amplitude_env
df['Instantaneous Phase'] = inst_phase

#filtro al gradiente de fase

cutoff = 0.1
order = 4

b,a = signal.butter(order, cutoff, btype='low')
grad_phase = signal.filtfilt(b, a, diff_phase-mean_phase)

df['Gradient Phase'] = grad_phase

In [ ]:
df.head()

In [ ]:
start_end_defectos = [
    [25014, 25042],
    [27460, 27487],
    [35630, 35658],
    [37207, 37236],
    [48343, 48372],
    [57421, 57450],
    [61722, 61750],
    [78988, 79016],
    [82306, 82336],
    [84344, 84373],
    [97845, 97875], #10
    [99883, 99914], 
    [101083, 101111],
    [131884, 131912],
    [165590, 165619],
    [166234, 166264],
    [183081, 183109],
    [187677, 187705],
    [203502, 203530],
    [219482, 219510],
    [235509, 235537], #20
    [275603, 275633],
    [277908, 277936],
    [301722, 301753],
    [322041, 322070],
    [322422, 322450],
    [339137, 339166],
    [393906, 393935],
    [395396, 395424],
    [396716, 396746],
    [412940, 412969], #30
    [413358, 413386], 
    [430261, 430289],
    [444404, 444433],
    [453962, 453990],
    [456065, 456095],
    [456863, 456892],
    [457187, 457215],
    [457390, 457419],
    [470756, 470783],
    [474540, 474568], #40
    [502372, 502400],
    [564199, 564227]
]

### **Ventanas**

Por EDA se obtiene una posible escala para la división de la ventana de tiempo. Esta corresponde entre 10.000 y 13.500 pts por ventana.
Por lo mismo, las ventanas se harán de 10.000pts con un overlap de 1.500 pts por lado.

In [ ]:
def create_windows(df, size = 4000, overlap = 500):
    num_windows = (len(df) - size) // (size - overlap) + 1
    windows = []
    for i in range(num_windows):
        start = i * (size - overlap)
        end = start + size
        window = df.iloc[start:end]
        windows.append(window)
    return windows

In [ ]:
def find_windows(start_end_defectos, windows):
    result = []
    for start, end in start_end_defectos:
        for i, window in enumerate(windows):
            if start >= window.index[0] and end <= window.index[-1]:
                result.append(i)
                break
    return result


In [ ]:
windows = create_windows(df)
window_indices = find_windows(start_end_defectos, windows)

### **frames for training**

In [ ]:
# from windows to dataframe
def data_for_model(windows):
    wframe = []
    for window in windows:
        a = window['Gradient Phase'].values
        b = window['Amplitude Envelope'].values
        c = np.concatenate((a, b), axis=0)
        wframe.append(c)

    wframe = pd.DataFrame(wframe)
    return wframe

In [ ]:
wframe = data_for_model(windows)
unique_indices = np.unique(window_indices)
wframe['label'] = np.where(wframe.index.isin(unique_indices), 1, 0)
wframe_no_labels = wframe.drop('label', axis=1)


### **SVM**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Split the dataframe into 70% training and 30% testing subsets
train_df, test_df = train_test_split(wframe, test_size=0.3, random_state=42)

# Create an instance of SVC
svm = SVC()

# Fit the model to the training data
svm.fit(train_df.drop('label', axis=1), train_df['label'])

# Predict the labels for the testing data
predictions = svm.predict(test_df.drop('label', axis=1))

# Print the shapes of the subsets
print("Training subset shape:", train_df.shape)
print("Testing subset shape:", test_df.shape)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Compute the confusion matrix
cm = confusion_matrix(test_df['label'], predictions, normalize='true')

# Plot the confusion matrix
sns.heatmap(np.round(cm, 3), annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Print the classification report
report = classification_report(test_df['label'], predictions)
print(report)


In [ ]:
wrong_indices = test_df.index[test_df['label'] != predictions]
print(wrong_indices)

In [ ]:
for ind in wrong_indices:
    subset = windows[ind]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=subset.index, y=subset['Amplitude Envelope'], name='Amplitude Envelope'))
    fig.add_trace(go.Scatter(x=subset.index, y=subset['Gradient Phase'], name='Gradient Phase'))
    fig.add_trace(go.Scatter(x=subset.index, y=subset['Filtered Signal'], name='Signal'))
    fig.update_layout(title='Index: ' + str(ind))
    fig.show()
    

En general, algoritmo se equivoca en zonas con 2 o más defectos.

# **Probar en otra señal**

In [ ]:
#load data

fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/"
fname = "datosconsu_021023_medio_1.mat"
cutoff = [8/1000, 11/1000]

set =  ds.MatFileToDataFrame(fpath, fname)
df_medios = set.get_dataframe(cutoff)
df.tail()

In [ ]:
amplitude_env = np.abs(df_medios['Hilbert Transform'])
inst_phase =  np.unwrap(np.angle(df_medios['Hilbert Transform']))
inst_freq = (np.diff(inst_phase)/(2.0*np.pi)*1000)
diff_phase =  np.diff(inst_phase)
mean_phase = np.mean(diff_phase)
diff_phase = np.insert(diff_phase, 0, 0)  
df_medios['Amplitude Envelope'] = amplitude_env
df_medios['Instantaneous Phase'] = inst_phase

#filtro al gradiente de fase

cutoff = 0.1
order = 4

b,a = signal.butter(order, cutoff, btype='low')
grad_phase = signal.filtfilt(b, a, diff_phase-mean_phase)

df_medios['Gradient Phase'] = grad_phase

In [ ]:
windows_medios = create_windows(df_medios)
test_medios = data_for_model(windows_medios)

In [ ]:
svm.predict(test_medios)
print(svm.predict(test_medios))

claramente no funciona para otra señal, ya que el modelo está entrenado para la señal previa. Se puede agregar los defectos de esta señal a mano, para mejorar el rendimiento, sin embargo no asegura que vaya a funcionar a mayor escala. Lo bueno, es que por ahora la regla de decisión, al menos visual, esta bien definida, por lo que aumentar la data no es dificil.
Para mejores resultados, hay que agregar más data, ajustar los hiperparametros de la red y ver otros modelos (red neuronal, para agregar dimesionalidad)

In [ ]:
min_amp = np.min(df_medios['Amplitude Envelope'])
indices =  df_medios[df_medios['Amplitude Envelope'] <= min_amp*30].index
len(indices)


In [ ]:
#peaks

peak, peak_info = signal.find_peaks(np.abs(df_medios['Gradient Phase']), height=[0, 5.0], distance=10)

print(f'Nº de peaks: \t {len(peak)}')
print(f'Amplitud promedio: \t {np.mean(peak_info["peak_heights"])}')
print(f'Amplitud máxima: \t {np.max(peak_info["peak_heights"])}')
print(f'Amplitud mínima: \t {np.min(peak_info["peak_heights"])}')
print(f'Amplitud STD: \t {np.std(peak_info["peak_heights"])}')

In [ ]:
#intersección
intersection = np.intersect1d(indices, peak, assume_unique=False, return_indices=False)
len(intersection)

In [ ]:
intersection

### **plot defectos medios**

In [ ]:
for k, i in enumerate(intersection[:10]):
   fig =  go.Figure()
   #fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = amp_filtered[i-500:i+500], name='AE Filtrada'))#, line_shape='linear'))
   fig.add_trace(go.Scatter(x = df_medios.index[i-500:i+500], y = (df_medios['Filtered Signal'][i-500:i+500]), name='Filtered Signal'))#, line_shape = 'linear'))']))
   fig.add_trace(go.Scatter(x = df_medios.index[i-500:i+500], y = (np.abs(df_medios['Gradient Phase'][i-500:i+500])), name='Gradiente Fase'))
   fig.add_trace(go.Scatter(x = df_medios.index[i-500:i+500], y = df_medios['Amplitude Envelope'][i-500:i+500], name='AE'))#, line_shape='linear'))
   #fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = amp_filtered2[i-500:i+500], name='AE Filtrada2'))
   fig.update_layout(title = f'Peak indice {i}, curva {k}')
   fig.show()


In [ ]:
min1_medios = []
min2_medios = []

for i in intersection:
    min1 = np.argmin(np.abs(df_medios['Gradient Phase'][i-15:i]))
    min2 = np.argmin(np.abs(df_medios['Gradient Phase'][i:i+15]))
    min1_medios.append(min1+i-15)
    min2_medios.append(min2+i)

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


In [ ]:
indices_tau = []
for i in range(len(intersection)):
    x = intersection[i]
    y = min2_medios[i]

    num1 = np.abs(df_medios['Gradient Phase'][x])/2
    subset = np.abs(df_medios['Gradient Phase'][x:y])
    num2 = find_nearest(subset, num1)

    indice2 = subset[np.abs(subset == num2)].index[0]
    indices_tau.append(indice2)

    

In [ ]:
df_medios_tiempos = pd.DataFrame()
df_medios_tiempos['peak'] = intersection

df_medios_tiempos['inicio_peak'] = min1_medios
df_medios_tiempos['fin_peak'] = min2_medios
df_medios_tiempos['duration'] = df_medios_tiempos['fin_peak'] - df_medios_tiempos['inicio_peak']
df_medios_tiempos['duration_seg'] = df_medios_tiempos['duration']/1000

df_medios_tiempos['peak/2'] = indices_tau
df_medios_tiempos['tau_samples'] = df_medios_tiempos['peak/2'] - df_medios_tiempos['peak']
df_medios_tiempos['tau_seg'] = df_medios_tiempos['tau_samples']/1000

df_medios_tiempos['difference'] = df_medios_tiempos['inicio_peak'].shift(-1) - df_medios_tiempos['fin_peak']




In [ ]:
df_medios_tiempos[['duration', 'tau_samples', 'difference']].describe()

Entonces, se pueden hacer ventanas de 2.000 puntos cada una, con un overlap de 500 puntos. Con esto me aseguro de que menos del 30% de las ventanas con defectos presenten defectos dobles y las ventanas que presentan defectos dobles, se puede dividir en dos y se les hace padding para rellenar y obtener 3 datos de esa, que correspondería a al ventana con defectos dobles, ventan con defecto 1 y ventana con defecto 2.


## **decomposición de las señales**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq


#### **amplitud media**

In [ ]:
mid_decompose = seasonal_decompose(df_medios['Amplitude Envelope'], model='aditive', period=1000)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = mid_decompose.trend.index, y = mid_decompose.trend, name='Trend'))
fig.add_trace(go.Scatter(x = mid_decompose.seasonal.index, y = mid_decompose.seasonal, name='Seasonal'))
fig.add_trace(go.Scatter(x = mid_decompose.resid.index, y = mid_decompose.resid, name='Residual'))
fig.update_layout(title = 'Descomposición de la señal')
fig.show()

In [ ]:
season_mid = mid_decompose.seasonal.values
fft_season_mid = fft(season_mid)
sampling_rate = 1000.0
N = len(season_mid)

normalize = N/2.0

freq_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(fft_season_mid)/normalize
normalize = N/2.0

freq_axis = fftfreq(N, d = 1.0/sampling_rate)
norm_amplitude = np.abs(fft_season_mid)/normalize


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_axis, y=norm_amplitude, mode='lines'))
fig.update_layout(title='Espectro AE estacional', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()

La onda envolvente, presenta una fuerte estacionalidad, que tiene como frecuencias dominantes de 1 a 7 Hz. 

#### **amplitud baja**

In [ ]:
low_decompose = seasonal_decompose(df['Amplitude Envelope'], model='aditive', period=1000)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = low_decompose.trend.index, y = low_decompose.trend, name='Trend'))
fig.add_trace(go.Scatter(x = low_decompose.seasonal.index, y = low_decompose.seasonal, name='Seasonal'))
fig.add_trace(go.Scatter(x = low_decompose.resid.index, y = low_decompose.resid, name='Residual'))
fig.update_layout(title = 'Descomposición de la señal (baja amplitud)')
fig.show()

In [ ]:

fft_season_low = fft(low_decompose.seasonal.values)
sampling_rate = 1000.0
N = len(low_decompose.seasonal.values)

normalize = N/2.0

freq_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(fft_season_low)/normalize
normalize = N/2.0

freq_axis = fftfreq(N, d = 1.0/sampling_rate)
norm_amplitude = np.abs(fft_season_low)/normalize


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_axis, y=norm_amplitude, mode='lines'))
fig.update_layout(title='Espectro AE estacional', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()